# Алгебраические коды

## Терминология

Пусть есть некоторое подпространство $\mathbb{W}$ пространства $\mathbb{C}$ над полем $\mathbb{F}$, а так же есть отображение $\operatorname{encoding} : \mathbb{W} \rightarrow \mathbb{C}$. Тогда будем называть алгебраическим (блоковым) кодом $\mathcal{C} = \operatorname{encoding}(\mathbb{W})$.

Линейный блоковый код это блоковый код, преобразующий $k$-мерное подпространство $\mathbb{F}_q^k$ в элементы $n$-мерного векторное пространства $\mathbb{F}_q^n$ над полем $\mathbb{F}_q$. Такой код задаётся порождающей матрицей $\underset{k \times n}{G}$. Иными словами, $\operatorname{encoding}(w) = w \cdot G$.

### Информационная совокупность

Пусть у нас есть некоторое подпространство $\mathcal{S}$ пространства $\mathcal{C}$, определяемое некоторой структурой $J$, называемой проектором. Тогда будем называть проекцией $c \in \mathcal{C}$ в $s \in \mathcal{S}$ такое преобразование, что $J(c) = s$. В случае линейных пространств $J$ это всего лишь множество координат $\{j_1, j_2, \dots j_k\}$, а $J(c) = \{c_{j_1}, c_{j_2}, \dots c_{j_k}\}$. Дополняющим проектором будем называть проектор $\bar{J}$ такой, что $|\bar{J}| + |J| = |\mathcal{C}|$ и $\bar{J} \cap J = \varnothing$.

Информационная совокупность кода $\mathcal{C}$ над полем $\mathbb{F}$ это такой минимальный проектор $J$, что для любого $f \in \mathbb{F}^{|J|}$ существует единственное кодовое слово $c \in \mathcal{C}$ такое, что $J(c) = f$. Тогда проверочной совокупностью называется проектор $\bar{J}$.

In [1]:
import numpy as np

from algebraic.matrix import project

c = np.array([1, 2, 4, 8, 16, 32, 64, 128])
j = np.array([2, 5, 1, 0])
f = project(c, j)
print(f)

[ 4 32  2  1]


## Декодирование

Рассмотрим общие методы декодирования алгебраических кодов

### Декодирование по информационным совокупностям

Пусть $y = c + e$, где $c$ — исходный вектор, $y$ — принятый вектор (жёсткое решение), а $e$ — вектор ошибки. Информационная совокупность $J$ называется свободной от ошибок, если для всех $i \in J$ верно $e_i = 0$. Если её удалось найти, то можно восстановить $c$ (исходя из определения $J$).

Общий алгоритм декодирования следующий. Пройдёмся по всем возможным $J$. Вычислим для каждой $J$ восстановленный вектор $c_J$. Результатом будет являться $\underset{c_J}{argmin} d_H(c_J, y)$, т.е. декодирование происходит по критерию минимального расстояния Хэмминга.

Для линейных кодов каждая возможная информационная совокупность $J$ соответствует некоторой матрице $G_J$, полученной из порождающей матрицы $G$ путём комбинации линейных преобразований над строками, такой, что столбцы с позициями из $J$ образуют единичную матрицу в $G_J$. Тогда $c_J = J(y) \cdot G_J$.

Далее будет показан пример перебора таких информационных совокупностей.

In [2]:
from algebraic.binary import binary_array

g = binary_array([[1] * 8 + [0] * 8, ([1] * 4 + [0] * 4) * 2, ([1] * 2 + [0] * 2) * 4, [1, 0] * 8, [1] * 16])
print("g:")
print(g)

def generate_j(g_j):
    n, m = len(g_j), len(g_j[0])
    j = [None] * n
    for i in range(0, m):
        if sum([1 if g_j[k][i] == 1 else 0 for k in range(n)]) == 1:
            j[max([k if g_j[k][i] == 1 else 0 for k in range(n)])] = i
    return j

y = binary_array([1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0])
print("y: " + str(y))

g:
[[1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]
y: [1 1 0 0 0 0 0 1 1 1 0 0 1 0 1 0]


In [3]:
from algebraic.binary import to_array

g_1 = np.copy(g)
g_1 += np.array([g_1[0] if i != 0 else [0] * len(g_1[0]) for i in range(len(g_1))])
g_1[0] += g_1[3]
g_1[0] += g_1[2]
g_1[3] += g_1[1]
print("g_1:")
print(g_1)

j_1 = generate_j(g_1)
y_j_1 = project(y, j_1)
print("j_1: " + str(j_1))
print("y_j_1: " + str(y_j_1))

c_1 = y_j_1 @ g_1
e_1 = y - c_1
print("c_1: " + str(c_1))
print("e_1: " + str(e_1))
print("d_1: " + str(sum(to_array(e_1))))

g_1:
[[1 0 0 1 1 0 0 1 0 1 1 0 0 1 1 0]
 [0 0 0 0 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 0 1 1 0 0 1 1 1 1 0 0 1 1 0 0]
 [0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]]
j_1: [0, 5, 2, 1, 15]
y_j_1: [1 0 0 1 0]
c_1: [1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0]
e_1: [0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 0]
d_1: 7


In [4]:
g_2 = np.copy(g_1)
g_2[0] += g_2[2]
g_2[1] += g_2[2]
print("g_2:")
print(g_2)

j_2 = generate_j(g_2)
y_j_2 = project(y, j_2)
print("j_2: " + str(j_2))
print("y_j_2: " + str(y_j_2))

c_2 = y_j_2 @ g_2
e_2 = y - c_2
print("c_2: " + str(c_2))
print("e_2: " + str(e_2))
print("d_2: " + str(sum(to_array(e_2))))

g_2:
[[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0]
 [0 0 1 1 0 0 1 1 1 1 0 0 1 1 0 0]
 [0 1 0 1 1 0 1 0 0 1 0 1 1 0 1 0]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]]
j_2: [0, 5, 7, 1, 15]
y_j_2: [1 0 1 1 0]
c_2: [1 1 0 0 0 0 1 1 0 0 1 1 1 1 0 0]
e_2: [0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 0]
d_2: 7


In [5]:
g_3 = np.copy(g_2)
g_3[3] += g_3[4]
g_3[2] += g_3[4]
print("g_3:")
print(g_3)

j_3 = generate_j(g_3)
y_j_3 = project(y, j_3)
print("j_3: " + str(j_3))
print("y_j_3: " + str(y_j_3))

c_3 = y_j_3 @ g_3
e_3 = y - c_3
print("c_3: " + str(c_3))
print("e_3: " + str(e_3))
print("d_3: " + str(sum(to_array(e_3))))

g_3:
[[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [0 0 1 1 1 1 0 0 0 0 1 1 1 1 0 0]
 [0 0 1 1 0 0 1 1 0 0 1 1 0 0 1 1]
 [0 1 0 1 1 0 1 0 1 0 1 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1]]
j_3: [0, 5, 7, 1, 9]
y_j_3: [1 0 1 1 1]
c_3: [1 1 0 0 0 0 1 1 1 1 0 0 0 0 1 1]
e_3: [0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1]
d_3: 3


### Декодирование методом порядковых статистик

Обобщим декодирование двоичных кодов по информационным совокупностям на случай метрики Евклида для мягких решений. Вместо $y$ на входе будем иметь вектор логарифмических отношений правдоподобия принятых символов $L$. Тогда оценим $\hat{c_i} = \delta(\operatorname{sign}(L_i), 1)$.

Далее будем пытаться наиболее надёжную информационную совокупность для $\hat{c}$. Сначала в качестве $J_0$ возьмём такую последовательность $(j_1, j_2, \dots j_k)$, что $|L_{j_i}| \ge |L_{j_{i+1}}|$, а $|L_{j_0}| = \max_i{L_i}$. То есть $J_0(L)$ образует последовательность первых $k$ порядковых статистик $L$.

Если $J$ не образует информационную совокупность, значит в столбце $j_{i'}$ в методе Гаусса не удалось получить единичное значение для диагональной матрицы. Тогда найдём наименьшее число $i'' > i"$, что можно продолжить метод Гаусса, и поменяем $j_{i'}$ на $j_{i''}$. В конце концов сформируется наиболее надёжная информационная совокупность $J_r$.

Однако $J_r$ может не оказаться свободной от ошибок. Тогда пройдёмся возможным векторам ошибок $e$ небольшого веса (например, не больше $t$), восстановим кодовое слово $c(e) = (J_r(\hat{c}) + e) \cdot G_{J_r}$ а по нему восстановим сигнал $L(e) = \operatorname{encoding}(c(e))$. Тогда в качестве ответа вернём кодовое слово $c = c(e_{best})$, где $e_{best} = \underset{e}{\operatorname{argmin}} d_E(L, L(e))$.